### Fetch response

In [ ]:
import pandas as pd

test_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

test_df.head()

In [ ]:
question_array = test_df.Question.to_list()

print(question_array)

#### CaLM-ADRD

In [ ]:
from test.evaluation import call_api_batch

answer_array = await call_api_batch(question_array)

print(answer_array)

In [4]:
test_df.loc[:, "adrd_qwen3_14b"] = answer_array

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")

In [ ]:
import pandas as pd

eval_df = pd.read_parquet("./data/66-test-dataset-with-adrd.parquet")

eval_df.head()

#### From OpenAI GPT

In [ ]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate

load_dotenv()

o3 = init_chat_model("o3-mini", model_provider="openai", temperature=1)


prompt = PromptTemplate(
    input_variables=["question"],
    template="""
You are a compassionate healthcare consultant specializing in caregiving for Alzheimer's Disease and Related Dementias
(ADRD). Your job is to provide empathetic, knowledgeable, and structured support to caregivers facing emotional,
practical, and medical challenges. You answer questions based on the provided context (user's input and chat history)
while offering responses that are warm, informative, and actionable.


# Thinking Process

Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most.

# Approach
	•	Answer starts with a paragraph (2-3 sentences) of introduction and emotional recognition and support, then answer the question in a structured way, and ends with a paragraph (2-3 sentences) of conclusion.
	•	Warm yet professional: Speak with kindness, avoiding overly clinical or detached language.
	•	Non-judgmental & supportive: Caregiving is challenging—reassure the user that they are doing their best.
	•	Use proper in text citations to reference the sources and support your statements. In format of "[<index>]" (e.g. "[1]", "[2]", "[3]", etc.).

# Context

The caregiver's current query:
{question}
""",
)

chain = prompt | o3

chain.invoke({"question": "Hello, how are you?"})

In [ ]:
from langchain_core.messages import BaseMessage

res_list = []

for idx, question in enumerate(question_array):
    response: BaseMessage = chain.invoke({"question": question})

    res_list.append(response.content)
    print(f"Processed {idx} questions")

print(res_list[:5])

In [6]:
test_df.loc[:, "gpt-o3"] = res_list

test_df.to_parquet("./data/66-test-dataset-with-adrd.parquet")